In [55]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression # 분류모델
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

log_df = pd.read_csv('20260116_142548_web_server_logs_2.csv')

# 전처리
# 타임 스탬프에서 시간 정보 추출
log_df['timestamp'] = pd.to_datetime(log_df['timestamp'])
log_df['hour'] = log_df['timestamp'].dt.hour # 다 같은 날이기 때문에 시간 정보만 추출

# status_code에서 is_error label 생성
log_df['is_error'] = log_df['status_code'].apply(lambda x: 1 if x >= 400 else 0)

# size를 로그로 변환
log_df['log_size'] = np.log1p(log_df['size'])  # log(1 + size)

y = log_df["label"]

# timestamp/size는 파생변수로 대체했으니 제거
X = log_df.drop(columns=["label", "timestamp", "size"])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 전처리 파이프라인
cat_cols = ["method"]
num_cols = ["hour", "status_code", "log_size", "is_error"]

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols), # 수치형 변수 표준화
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols), # 범주형 변수 원-핫 인코딩
    ],
    remainder="drop"
)

# 모델
model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("clf", LogisticRegression(max_iter=2000, class_weight="balanced"))
])

# 학습/평가
model.fit(X_train, y_train)
pred = model.predict(X_test)

print("Accuracy :", accuracy_score(y_test, pred))
print("Precision:", precision_score(y_test, pred, zero_division=0))
print("Recall   :", recall_score(y_test, pred, zero_division=0))
print("F1-score :", f1_score(y_test, pred, zero_division=0))
print("\n[classification_report]\n", classification_report(y_test, pred, digits=4))

Accuracy : 0.9166666666666666
Precision: 0.8131868131868132
Recall   : 0.9024390243902439
F1-score : 0.8554913294797688

[classification_report]
               precision    recall  f1-score   support

           0     0.9617    0.9220    0.9415       218
           1     0.8132    0.9024    0.8555        82

    accuracy                         0.9167       300
   macro avg     0.8875    0.9122    0.8985       300
weighted avg     0.9211    0.9167    0.9180       300

